In [ ]:
try:
    import matplotlib.pyplot as plt
    %matplotlib inline
    %config InlineBackend.figure_format = 'svg'
except ImportError:
    print('Matplotlib not installed. Please use pip(3) to install required package!')

try:
    import numpy as npy
except ImportError:
    print('Numpy not installed. Please use pip(3) to install required package!')
    
try:
    from cvxpy import *
except ImportError:
    print('CVXPy not installed. Please use pip(3) to install required package!')

class TIConstraints:
    a_min = -8
    a_max = 8
    s_min = 0
    s_max = 100
    v_min = 0
    v_max = 10
    

def plot_state_vector(x : Variable, c : TIConstraints, s_obj = None):
    N = x.size[1]-1
    s_max = npy.ceil(npy.amax(x[0,:].value.A.flatten())*1.1/10)*10
    # Plot (u_t)_1.
    #ax = f.add_subplot(411)
    #plt.plot(u.value.A.flatten())
    #plt.ylabel(r"$(u_t)_1$", fontsize=16)
    #plt.yticks(np.linspace(a_min, a_max, 3))
    #plt.xticks([])

    # Plot (x_t)_1.
    plt.subplot(4,1,1)
    x1 = x[0,:].value.A.flatten()
    plt.plot(npy.array(range(N+1)),x1)
    if s_obj is not None:
        plt.plot(npy.array(range(N+1)),s_obj)
    plt.ylabel(r"$s$", fontsize=16)
    plt.yticks(npy.linspace(c.s_min, s_max, 3))
    plt.ylim([c.s_min, s_max])
    plt.xticks([])

    # Plot (x_t)_2.
    plt.subplot(4,1,2)
    x2 = x[1,:].value.A.flatten()
    plt.plot(npy.array(range(N+1)),x2)
    plt.yticks(npy.linspace(c.v_min,c.v_max,3))
    plt.ylim([c.v_min, c.v_max+2])
    plt.ylabel(r"$v$", fontsize=16)
    plt.xticks([])

    # Plot (x_t)_3.
    plt.subplot(4,1,3)
    x2 = x[2,:].value.A.flatten()
    plt.plot(npy.array(range(N+1)),x2)
    plt.yticks(npy.linspace(c.a_min,c.a_max,3))
    plt.ylim([c.a_min, c.a_max+2])
    plt.ylabel(r"$a$", fontsize=16)
    plt.xticks([])

    # Plot (x_t)_4.
    plt.subplot(4,1,4)
    x2 = x[3,:].value.A.flatten()
    plt.plot(npy.array(range(N+1)), x2)
    plt.yticks(npy.linspace(c.a_min,c.a_max,3))
    plt.ylim([c.a_min, c.a_max])
    plt.ylabel(r"$j$", fontsize=16)
    plt.xticks(range(0,N+1))
    plt.xlabel(r"$k$", fontsize=16)
    plt.tight_layout()
    plt.show()

In [ ]:
#---------------
# EXERCISE 5
# Modify optimization horizon, step size, and weights of the problem
#---------------

# problem data
N  = 20  # No of time steps (horizon)
n  = 4   # length of state vector 
m  = 1   # length of input vector
dT = 0.5 # time step

# set up variables
x = Variable(n,N+1) # optimization vector x contains n states per time step 
u = Variable(m,N) # optimization vector u contains 1 state

# set up constraints
c = TIConstraints
c.a_min = -8 # Minimum feasible acceleration of vehicle
c.a_max = 8 # Maximum feasible acceleration of vehicle
c.s_min = 0 # Minimum allowed position
c.s_max = 100 # Maximum allowed position
c.v_min = 0 # Minimum allowed velocity (no driving backwards!)
c.v_max = 15 # Maximum allowed velocity (speed limit)
tiConstraints = []

# weights for cost function
w_s = 0
w_v = 5
w_a = 1
w_j = 3
Q = npy.eye(n)*npy.transpose(npy.array([w_s,w_v,w_a,w_j]))
w_u = 1
R = w_u

#---------------
# EXERCISE 1
# set up system matrices
#---------------

A = npy.array()
B = npy.array()

In [ ]:
# initial state of vehicle
x_0 = npy.array([[0],[0],[0],[0]]) # initial state is [0,0,0,0]

# reference velocity
v_ref = 27.766

# Set up optimization problem
states = []
for k in range(N):
    # cost function
    cost = quad_form(x[:,k+1]-npy.transpose(npy.array([0,v_ref,0,0])),Q) + quad_form(u[:,k],R) #sum_squares(x[:,k+1]) + sum_squares(u[:,k])
    # time variant state and input constraints
    constr = [x[:,k+1] == A*x[:,k] + B*u[:,k]]
    states.append( Problem(Minimize(cost), constr) )
    
# sums problem objectives and concatenates constraints.
prob = sum(states)
# constraints for all states and inputs
prob.constraints += tiConstraints
# initial state and goal state constraints
prob.constraints += [x[:,0] == x_0]

# Solve optimization problem
prob.solve(verbose=True)
print("Problem is convex:",prob.is_dcp())
print("Problem solution is "+prob.status)

In [ ]:
# plot results
plot_state_vector(x, c)

In [ ]:
#---------------
# EXERCISE 2
# Set up constraints of the problem to fix infeasibility
#---------------
tiConstraints 

# Adjust problem
prob.constraints += tiConstraints

# Solve optimization problem
prob.solve()
print("Problem is convex:",prob.is_dcp())
print("Problem solution is "+prob.status)

# plot results
plot_state_vector(x, c)

In [ ]:
#---------------
# EXERCISE 4
# Modify initial state to v=8 and v=10, explain the results and propose a solution!
#---------------
# initial state of vehicle
x_0 = npy.array([[0],[5],[0],[0]]) # initial state is [0,5,0,0]

# reference velocity
v_ref = 11

# obstacle
s_o = 8.0
v_o = 4
dS = 2
s_obj = s_o + npy.array(range(0,N+1))*dT*v_o - dS

#---------------
# EXERCISE 3
# Integrate obstacle into constraints of optimization problem
#---------------

# Set up optimization problem
states = []
for k in range(N):
    # cost function
    cost = quad_form(x[:,k+1]-npy.transpose(npy.array([0,v_ref,0,0])),Q) + quad_form(u[:,k],R) #sum_squares(x[:,k+1]) + sum_squares(u[:,k])
    # single state and input constraints
    constr = [x[:,k+1] == A*x[:,k] + B*u[:,k]]
    states.append( Problem(Minimize(cost), constr) )
    
# sums problem objectives and concatenates constraints.
prob = sum(states)
# constraints for all states and inputs
prob.constraints += tiConstraints
# initial state and goal state constraints
prob.constraints += [x[:,0] == x_0]

# Solve optimization problem
prob.solve(verbose=True)
print("Problem is convex:",prob.is_dcp())
print("Problem solution is "+prob.status)

# plot results
plot_state_vector(x, c, s_obj + dS)